In [1]:
import ROOT
import subprocess
import edm4hep
from podio import root_io

In [2]:

whizard_prefix = subprocess.run(['whizard-config', '--prefix'], capture_output=True, encoding='ascii').stdout.strip()
whizard_libs = f"{whizard_prefix}/lib/"
print(whizard_libs)

/cvmfs/sw-nightlies.hsf.org/key4hep/releases/2025-05-29/x86_64-almalinux9-gcc14.2.0-opt/whizard/3.1.4-bkajy7/lib/


In [3]:
# ROOT.gInterpreter.Declare(f"R__ADD_LIBRARY_PATH({whizard_libs})")
# ROOT.gInterpreter.Declare("R__LOAD_LIBRARY(libomega_core.so.0)")
ROOT.gSystem.AddDynamicPath(whizard_libs)
ROOT.gSystem.Load("libomega_core.so")
ROOT.gSystem.Load("libwhizard.so")
ROOT.gSystem.Load("libwhizard_main.so")
ROOT.gSystem.Load("libomega.so")
ROOT.gSystem.Load("whizard/cc20_ac_inclusive/.libs/default_lib.so")

0

In [4]:
ROOT.gInterpreter.Declare("#include \"test_whizard.h\"")

True

In [5]:
from model_parser import ModelParser
model_parser = ModelParser("SM_ac.mdl")

# add derivation of lz and kz according to lep parametrisation
model_parser.add_derived_parameter("lz", "la")
model_parser.add_derived_parameter("kz", "1.0 - (ka - 1.0) * sw**2/cw**2 + (g1z - 1.0)")

pars = model_parser.get_parameters_list()
print(pars)
ROOT.init_parameters(pars)

model_parser.set_parameters({
    "g1z": 1.001,
    "ka": 1.000,
    "la": 0.000,
    })
pars_alt1 = model_parser.get_parameters_list()
print(pars_alt1)

model_parser.set_parameters({
    "g1z": 1.000,
    "ka": 1.001,
    "la": 0.000,
    })
pars_alt2 = model_parser.get_parameters_list()
print(pars_alt2)

model_parser.set_parameters({
    "g1z": 1.000,
    "ka": 1.000,
    "la": 0.001,
    })
pars_alt3 = model_parser.get_parameters_list()
print(pars_alt3)

[1.16639e-05, 91.1882, 80.419, 125.0, 0.1178, 0.000511, 0.1057, 1.777, 0.12, 1.25, 4.2, 174.0, 1.523, 2.443, 2.049, 0.004143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 5000.0, 0.0, 0.0, 0.0, 0.0, 246.21845810181634, 0.8819013863635865, 0.4714339240338821, 0.30795615429614365, 0.0, 0.0, 0.0, 132.50494581248503]
[1.16639e-05, 91.1882, 80.419, 125.0, 0.1178, 0.000511, 0.1057, 1.777, 0.12, 1.25, 4.2, 174.0, 1.523, 2.443, 2.049, 0.004143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.001, 0.0, 0.0, 0.0, 0.0, 1.0, 1.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 5000.0, 0.0, 0.0, 0.0, 0.0, 246.21845810181634, 0.8819013863635865, 0.4714339240338821, 0.30795615429614365, 0.0, 0.0, 0.0, 132.50494581248503]
[1.16639e-05, 91.1882, 80.419, 125.0, 0.1178, 0.000511, 0.1057, 1.777, 0.12, 1.25, 4.2, 174.0, 1.523, 2.443, 2.049, 0.004143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.001, 0.999714

In [6]:
reader = root_io.Reader("whizard/cc20_ac_inclusive_ISR/ww.1k.edm4hep.root")

In [7]:
events = reader.get("events")
event = events[17]

In [8]:
mcparticles = event.get("MCParticle")

In [9]:
parameter_names = ["sqme", "sqme_alt1", "sqme_alt2", "sqme_alt3", "weight_alt1", "weight_alt2", "weight_alt3"]
parameters_whizard = {}
for name in parameter_names:
    parameters_whizard[name] = event.get_parameter(name)
print(parameters_whizard)

{'sqme': 5.619044054691151e-06, 'sqme_alt1': 5.620592500435446e-06, 'sqme_alt2': 5.6190498806572235e-06, 'sqme_alt3': 5.619517867226903e-06, 'weight_alt1': 1.0002755710276023, 'weight_alt2': 1.0000010368251282, 'weight_alt3': 1.0000843226234106}


In [10]:
data = []
keep_list = [
    # 0, # e- pre-ISR
    # 1, # e+ pre-ISR
    2, # e- post-ISR
    3, # e+ post-ISR
    # 4, # ISR gamma 1
    # 5, # ISR gamma 2
    6, # e+-
    7, # nu
    8, # u
    9 # d
]
for i, mcp in enumerate(mcparticles):
    if not i in keep_list:
        continue
    data.append(mcp.getEnergy())
    momentum = mcp.getMomentum()
    data.append(momentum.x)
    data.append(momentum.y)
    data.append(momentum.z)

In [11]:
ROOT.new_event(data)
sqme1 = ROOT.sqme(1)
sqme2 = ROOT.sqme(2)
print(sqme1/parameters_whizard["sqme"])
print(sqme2/parameters_whizard["sqme"])
print((sqme1+sqme2)/parameters_whizard["sqme"])

14.257648245421365
3.652985294495531
17.910633539916898


In [12]:
ROOT.new_event(data, pars_alt1)
sqme1_alt1 = ROOT.sqme(1)
sqme2_alt1 = ROOT.sqme(2)
print((sqme1_alt1+sqme2_alt1)/parameters_whizard["sqme_alt1"])
print(sqme1_alt1/sqme1)
print(sqme2_alt1/sqme2)

17.910633540258328
1.0005687673575894
0.9991312221016505


In [13]:
print(f"sqme: {sqme1+sqme2}, sqme1: {sqme1}, sqme2: {sqme2}")
print(f"sqme_alt1: {sqme1_alt1+sqme2_alt1}, sqme1_alt1: {sqme1_alt1}, sqme2_alt1: {sqme2_alt1}")
print(f"weight: {1.0}, weight_alt1: {parameters_whizard['weight_alt1']}")
print(f"sqme_alt1/sqme: {(sqme1_alt1 + sqme2_alt1) / (sqme1 + sqme2)}")
print(f"sqme1_alt1/sqme1: {(sqme1_alt1) / (sqme1)}")
print(f"sqme2_alt1/sqme2: {(sqme2_alt1) / (sqme2)}")

sqme: 0.00010064063890822195, sqme1: 8.011435360731263e-05, sqme2: 2.0526285300909318e-05
sqme_alt1: 0.00010066837255442351, sqme1_alt1: 8.015992003651884e-05, sqme2_alt1: 2.0508452517904673e-05
weight: 1.0, weight_alt1: 1.0002755710276023
sqme_alt1/sqme: 1.0002755710466709
sqme1_alt1/sqme1: 1.0005687673575894
sqme2_alt1/sqme2: 0.9991312221016505
